<a href="https://colab.research.google.com/github/Vishal-ib97/Developer-Name-Disambiguation/blob/main/data_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from IPython.core.display import HTML
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import os
import re
import pandas as pd
import numpy as np
import sys
import re
import unicodedata

In [2]:
#mount drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#json files with contributor info
#dataframe containing the user ids and the names
import json
import urllib
url = "http://people.apache.org/public/icla-info.json"
json_url = urllib.request.urlopen(url)
data2 = json.loads(json_url.read())
com_names2 = pd.DataFrame(list(data2['committers'].items()),columns = ['user_ids','author_name'])
com_names2.to_csv('committers_with_ids.csv')

#dataframe containing the user ids and project details for every project
import json
import urllib
url = "http://people.apache.org/public/public_ldap_projects.json"
json_url = urllib.request.urlopen(url)
data = json.loads(json_url.read())

com_names = pd.DataFrame(columns = ['createTimestamp', 'modifyTimestamp', 'members', 'owners', 'pmc', 'podling', 'project'])
for proo in data["projects"].keys():
  if 'project' not in data["projects"][proo]:
    data["projects"][proo]["project"] = proo
  com_names = com_names.append(data["projects"][proo], ignore_index = True)

com_names.drop(['officer', 'owners'], axis = 1, inplace = True)
com_names = com_names.explode('members')
com_names.rename(columns = {'members' : 'user_ids'}, inplace = True)
com_names.to_csv('ids_with_project_deets.csv')

com = com_names2.merge(com_names, on = 'user_ids')
com.author_name = com.author_name.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
com.author_name = com.author_name.str.lower()
com.author_name = com.author_name.astype('str')
com.user_ids = com.user_ids.astype('str')
com.drop_duplicates(subset = ['author_name', 'user_ids', 'project'], inplace = True)
replace_str = [ ",", "\):", "/", "\\", "*", "$", "`", "\'", "\"", "|", ",", " \)", "=", "+", "#", "=", "\[", "\]", "?", "\(\)\;", "\);", ";", ".", "-"]
for i in replace_str:
  com['author_name'] = com['author_name'].str.replace(i,"")
com.to_csv("/content/gdrive/MyDrive/GG_Project_NLP/Data/commit_messages/json_data_updated.csv")

In [ ]:
#accumulating the from-subject data from the messages taken from all the pages
f = pd.DataFrame()
main = "/content/gdrive/MyDrive/GG_Project_NLP/Data/commit_messages/subject_from"
for i in os.listdir(main):
  path = os.path.join(main, i)
  f = pd.concat([f, pd.read_csv(path)])
f.to_csv("/content/gdrive/MyDrive/GG_Project_NLP/Data/commit_messages/commit_data_from_subject_10.csv")

In [ ]:
#accumulating commit message data for all the projects
new_df = pd.DataFrame()
main = "/content/gdrive/MyDrive/GG_Project_NLP/Data/commit_messages/all_pages_author_email"
for i in os.listdir(main):
  path = os.path.join(main, i)
  new_df = pd.concat([new_df, pd.read_csv(path,low_memory=False)])

com = pd.read_csv("/content/gdrive/MyDrive/GG_Project_NLP/Data/commit_messages/json_data_updated.csv") #selecting projects that have user id details
new_df = new_df[new_df.proj_name.isin(com.project)]
new_df.drop_duplicates(subset = ['proj_name', 'date_time', 'author_name', 'email_id'], inplace = True) #dropping duplicates
new_df.to_csv("/content/gdrive/MyDrive/GG_Project_NLP/Data/commit_messages/new_commits_df.csv")

In [19]:
new_df.head()

,Unnamed: 0,proj_name,date_time,author_name,email_id,Unnamed: 0.1
1966,1966.0,accumulo,2020-12-02 21:41:00,dependabot[bot],<49699333+dependabot[bot]@users.noreply.github...,NaN
1967,1967.0,accumulo,2020-12-03 14:03:00,Christopher Tubbs,<ctubbsii@apache.org>,NaN
1968,1968.0,accumulo,2020-12-04 16:02:00,Dave Marion,<dlmarion@apache.org>,NaN
1969,1969.0,accumulo,2020-11-02 15:14:00,Mike Miller,<mmiller@apache.org>,NaN
1970,1970.0,accumulo,2020-11-03 00:53:00,Christopher Tubbs,<ctubbsii@apache.org>,NaN
